In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import (
    FewShotChatMessagePromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here I know this:
        Capital: Paris
        Language: French
        Food: Croissant, Baguette, Cheese, Wine
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        Here I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza, Pasta, Gelato,
        Currency: Euro
        """,
    },
    {
        "country": "Spain",
        "answer": """
        Here I know this:
        Capital: Madrid
        Language: Spanish
        Food: Paella, Tapas, Churros
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)
few_shot_example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. You give short answers about countries."),
        few_shot_example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat
chain.invoke({"country": "Mexico"})


        Here I know this:
        Capital: Mexico City
        Language: Spanish
        Food: Tacos, Guacamole, Tamales
        Currency: Mexican Peso
        

AIMessage(content='\n        Here I know this:\n        Capital: Mexico City\n        Language: Spanish\n        Food: Tacos, Guacamole, Tamales\n        Currency: Mexican Peso\n        ')